In [46]:
from dotenv import load_dotenv
import os
import requests
import time

import pandas as pd

api_key = os.environ.get('riot_api_key')
root_url = '.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/'

# Information 
regions = ['euw1','kr', 'na1'] # largest regions
ranks = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER'] # Ranks
divisions = ['IV', 'III', 'II', 'I'] # Divisions



def get_summoner_id_rank(region='euw1', rank='SILVER', division='I', page=1):
    '''Get summoner id from region rank division
    Arguments:
        region: Region. Default to euw1
        rank: Rank. Default to SILVER
        division: Division. Default to I
        page: Page. Default to 1
    Return:
        str: summonerID'''
    
    main_url = 'https://%s%s%s/%s?page=%d&api_key=%s' % (region, root_url, rank, division, page, api_key)

    while True: # While loop allows to use continue
        response = requests.get(main_url) #Requests information from the API

        if response.status_code == 429:  # To deal with api rate limit
            time.sleep(10)
            continue
        
        return response.json()

In [31]:
sum_id = pd.DataFrame(get_summoner_id_rank())[['summonerId', 'wins', 'losses']]
sum_id['rank'] = ('%s %s' % ('SILVER','I'))
sum_id

,summonerId,wins,losses,rank
0,6q81R03U5LdPA29s4ZQP-9YSHksBOYn7RcGRHjx4ek3XgIWj,11,7,SILVER I
1,2KfbhiPzzlanNyuiNjRRl6oxcc7-yE6IaeovNyVhX-7O92ow,9,13,SILVER I
2,v0PCKs1dQjdQwj4LBizZh6aR27SjatdF1yTKG1QUntS5oScH,45,37,SILVER I
3,GLA5RO5KX6KRie2Vmgt7DrSEcXx99NpAj3L0Nj4YkTl0hQLk,43,31,SILVER I
4,kzxsV4Reoz-c8eQki35RCLRCS6mvpMN0a0BEEusV6MXcXrpB,8,7,SILVER I
...,...,...,...,...
200,CRmL8xZB4VnfA7DNCKvAjVODOXGgQVrsvTDReSBMQTNxzcPb,5,11,SILVER I
201,Uxg-JNT89MqchnczEw_bJNUvd6ar2kmX73OI-CLBBTdfDD8A,4,2,SILVER I
202,nbB1JhSV2MONrHOosfeaZwayKSvc7zIO10KG6OtdsDH96v_t,4,3,SILVER I
203,fF8L7qa4SXERl_osPGvU6oCtRJkrRfQ2ntTRlDwlYsg-V8IX,22,8,SILVER I


In [62]:
# Generate a dataFrame
df = pd.DataFrame()
for page in range(1,6):
    for rank in  ranks:
        if rank in ['MASTER','GRANDMASTER','CHALLENGER']: # Starting from the MASTER rank there are no divisions
            if page > 1:
                continue
            else:
                summonerId_df = pd.DataFrame(get_summoner_id_rank(region='euw1', rank=rank, division='I', page=1))[['summonerId', 'wins', 'losses']]
                summonerId_df['ranks'] = ("%s %s" %(rank, "I"))
                df = pd.concat([df, summonerId_df[:200]])
                continue
        for division in divisions:
            summonerId_df = pd.DataFrame(get_summoner_id_rank(region='euw1', rank=rank, division=division, page=page))[['summonerId', 'wins', 'losses']]
            summonerId_df['ranks'] = ("%s %s" %(rank, division))
            df = pd.concat([df, summonerId_df[:200]])


print("DONE!!!!")

DONE!!!!


In [66]:
# Clear data
active_players = df[df['losses'] + df['wins'] > 100]
active_players = active_players.dropna()
active_players = active_players.rename(columns={0:"summonerId"}) 

In [70]:
# Create csv file
active_players[['summonerId']].to_csv('/Users/alekaamirtay/Desktop/Coding/my_projects/new_project/python_project/python_db/info_docs/summonerID_file.csv', index=False) # Write df as csv file